해당 평가는 Konlpy 설치가 번거로워서 구글 Colab에서 진행하였습니다.

In [1]:
!pip install rouge

In [2]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 25.7 MB/s eta 0:00:00


In [8]:
import pandas as pd
import ast

In [53]:
# 체크포인트 950
# df = pd.read_csv('/content/evaluation.csv')

# 체크포인트 1900
# df = pd.read_csv('/content/evaluation_ckpt_1900.csv')

# 체크포인트 2580
df = pd.read_csv('/content/evaluation_ckpt_2850.csv')

In [44]:
df

,관리번호,설비번호(구),설비명,수리일자,고장내용,고장부품,불량유형,조치내용,수리처,수리시간(분),수리금액,predictions,labels
0,Z000000001,Z000000001,기타,2023-01-03,3공장렉 철거후 2공장으로 이동,3공장 렉,철거,2공장으로 이동,오정수이승희,480,NaN,"{'고장부품': '3공장 철거', '불량유형': '철거 후 이동', '조치내용': ...","{'고장부품': '3공장 렉', '불량유형': '철거', '조치내용': '2공장으로..."
1,Z000000001,Z000000001,기타,2023-01-03,CN7라인 천정 전등 교체,CN7라인 천정 전등,고장,전등 교체,송명호윤동태,180,NaN,"{'고장부품': 'CN7라인 천정 전등', '불량유형': '고장', '조치내용': ...","{'고장부품': 'CN7라인 천정 전등', '불량유형': '고장', '조치내용': ..."
2,Z000000001,Z000000001,기타,2023-01-03,E4L 인너카바 리벳툴 진공이상 리벳 잔영물 제거 청소,E4L 인너카바 리벳툴,진공이상,리벳 잔영물 제거 청소,최혁진윤동태,30,NaN,"{'고장부품': 'E4L 인너카바 리벳', '불량유형': '진공이상', '조치내용'...","{'고장부품': 'E4L 인너카바 리벳툴', '불량유형': '진공이상', '조치내용..."
3,Z000000001,Z000000001,기타,2023-01-03,전착도장 대차 제작중,전착도장 대차,NaN,제작중,김정표오정수,140,NaN,"{'고장부품': '전착도장', '불량유형': '대차', '조치내용': '제작 중'}","{'고장부품': '전착도장 대차', '불량유형': nan, '조치내용': '제작중'}"
4,M000000038,ASM-029,조립기(DH 스위치 조립기),2023-01-04,#40 LH 트램스파 동작이상 전후진 솔밸브 단선 수리,#40 LH 트램스파 전후진 솔밸브,동작이상,단선 수리,송명호윤동태,150,NaN,"{'고장부품': '#40 LH 트램스파', '불량유형': '동작이상, 전후진 솔밸브...","{'고장부품': '#40 LH 트램스파 전후진 솔밸브', '불량유형': '동작이상'..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,Z000000001,Z000000001,기타,2023-06-03,포르비아시팅 라인 이관 작업,포르비아시팅 라인,이관 작업,이관 작업 진행,정은호오정수,300,NaN,"{'고장부품': '포르비아시팅 라인', '불량유형': '이관 필요', '조치내용':...","{'고장부품': '포르비아시팅 라인', '불량유형': '이관 작업', '조치내용':..."
796,S000000019,CNV-001,스크랩 컨베어,2023-06-05,"구동불가 주모터 반대 스크랩제거,아웃플레이트 1EA 설치,샤프트분할핀 파손교체 다량","주모터, 아웃플레이트, 샤프트분할핀","스크랩 이물질 결함, 아웃플레이트 누락, 샤프트분할핀 파손","스크랩 제거, 아웃플레이트 1EA 설치, 샤프트분할핀 교체",생기전원,200,NaN,"{'고장부품': '주모터, 샤프트', '불량유형': '구동불가, 파손', '조치내용...","{'고장부품': '주모터, 아웃플레이트, 샤프트분할핀', '불량유형': '스크랩 이..."
797,Z000000001,Z000000001,기타,2023-06-05,E4L 공박스 청소라인 선풍기 2대 설치,E4L 공박스 청소라인,설치 필요,선풍기 2대 설치,윤동탸오정수,100,NaN,"{'고장부품': 'E4L 공박스 청소라인', '불량유형': '설치 필요', '조치내...","{'고장부품': 'E4L 공박스 청소라인', '불량유형': '설치 필요', '조치내..."
798,Z000000001,Z000000001,기타,2023-06-05,수동리프트 발판 파손 용접 작업,수동리프트 발판,파손,용접 작업으로 복원,오정수윤동태,50,NaN,"{'고장부품': '수동리프트 발판', '불량유형': '파손', '조치내용': '용접...","{'고장부품': '수동리프트 발판', '불량유형': '파손', '조치내용': '용접..."


In [45]:
print(df['predictions'].loc[0])

{'고장부품': '3공장 철거', '불량유형': '철거 후 이동', '조치내용': '2공장으로 이동'}


In [46]:
print(df['labels'].loc[0])

{'고장부품': '3공장 렉', '불량유형': '철거', '조치내용': '2공장으로 이동'}


In [47]:
# 딕셔너리 형태의 문자열을 파싱하는 함수
def parse_dict_string(dict_str):
    """문자열 형태의 딕셔너리를 실제 딕셔너리로 변환"""
    if pd.isna(dict_str) or dict_str == '':
        return {}
    try:
        # 문자열이 이미 딕셔너리 형태인 경우
        if isinstance(dict_str, dict):
            return dict_str
        # 문자열을 딕셔너리로 변환
        return ast.literal_eval(dict_str)
    except:
        return {}

In [48]:
# predictions 열 분리
if 'predictions' in df.columns:
    # predictions 열의 각 딕셔너리에서 키들을 추출
    predictions_dicts = df['predictions'].apply(parse_dict_string)

    # 모든 고유한 키들을 찾기
    all_keys = set()
    for d in predictions_dicts:
        all_keys.update(d.keys())

    # 각 키별로 새로운 열 생성
    for key in all_keys:
        df[f'predictions_{key}'] = predictions_dicts.apply(lambda x: x.get(key, None))

# labels 열 분리
if 'labels' in df.columns:
    # labels 열의 각 딕셔너리에서 키들을 추출
    labels_dicts = df['labels'].apply(parse_dict_string)

    # 모든 고유한 키들을 찾기
    all_keys = set()
    for d in labels_dicts:
        all_keys.update(d.keys())

    # 각 키별로 새로운 열 생성
    for key in all_keys:
        df[f'labels_{key}'] = labels_dicts.apply(lambda x: x.get(key, None))

print("분리된 열들:")
print([col for col in df.columns if col.startswith(('predictions_', 'labels_'))])

분리된 열들:
['predictions_고장부품', 'predictions_불량유형', 'predictions_조치내용', 'labels_고장부품', 'labels_불량유형', 'labels_조치내용']


In [49]:
df.head()

,관리번호,설비번호(구),설비명,수리일자,고장내용,고장부품,불량유형,조치내용,수리처,수리시간(분),수리금액,predictions,labels,predictions_고장부품,predictions_불량유형,predictions_조치내용,labels_고장부품,labels_불량유형,labels_조치내용
0,Z000000001,Z000000001,기타,2023-01-03,3공장렉 철거후 2공장으로 이동,3공장 렉,철거,2공장으로 이동,오정수이승희,480,NaN,"{'고장부품': '3공장 철거', '불량유형': '철거 후 이동', '조치내용': ...","{'고장부품': '3공장 렉', '불량유형': '철거', '조치내용': '2공장으로...",3공장 철거,철거 후 이동,2공장으로 이동,3공장 렉,철거,2공장으로 이동
1,Z000000001,Z000000001,기타,2023-01-03,CN7라인 천정 전등 교체,CN7라인 천정 전등,고장,전등 교체,송명호윤동태,180,NaN,"{'고장부품': 'CN7라인 천정 전등', '불량유형': '고장', '조치내용': ...","{'고장부품': 'CN7라인 천정 전등', '불량유형': '고장', '조치내용': ...",CN7라인 천정 전등,고장,천정 전등 교체,CN7라인 천정 전등,고장,전등 교체
2,Z000000001,Z000000001,기타,2023-01-03,E4L 인너카바 리벳툴 진공이상 리벳 잔영물 제거 청소,E4L 인너카바 리벳툴,진공이상,리벳 잔영물 제거 청소,최혁진윤동태,30,NaN,"{'고장부품': 'E4L 인너카바 리벳', '불량유형': '진공이상', '조치내용'...","{'고장부품': 'E4L 인너카바 리벳툴', '불량유형': '진공이상', '조치내용...",E4L 인너카바 리벳,진공이상,리벳 잔영물 제거 및 청소,E4L 인너카바 리벳툴,진공이상,리벳 잔영물 제거 청소
3,Z000000001,Z000000001,기타,2023-01-03,전착도장 대차 제작중,전착도장 대차,NaN,제작중,김정표오정수,140,NaN,"{'고장부품': '전착도장', '불량유형': '대차', '조치내용': '제작 중'}","{'고장부품': '전착도장 대차', '불량유형': nan, '조치내용': '제작중'}",전착도장,대차,제작 중,None,None,None
4,M000000038,ASM-029,조립기(DH 스위치 조립기),2023-01-04,#40 LH 트램스파 동작이상 전후진 솔밸브 단선 수리,#40 LH 트램스파 전후진 솔밸브,동작이상,단선 수리,송명호윤동태,150,NaN,"{'고장부품': '#40 LH 트램스파', '불량유형': '동작이상, 전후진 솔밸브...","{'고장부품': '#40 LH 트램스파 전후진 솔밸브', '불량유형': '동작이상'...",#40 LH 트램스파,"동작이상, 전후진 솔밸브 단선",수리,#40 LH 트램스파 전후진 솔밸브,동작이상,단선 수리


In [50]:
from konlpy.tag import Okt
from rouge import Rouge

def calculate_rouge_for_korean_sentence(reference_sentence, hypothesis_sentence):
    # 형태소 분석기 초기화
    okt = Okt()

    # 형태소 분석을 통한 토큰화
    def tokenize(text):
        return ' '.join(okt.morphs(text))

    # 두 문장 토큰화
    tokenized_reference = tokenize(reference_sentence)
    tokenized_hypothesis = tokenize(hypothesis_sentence)

    # ROUGE 점수 계산
    rouge = Rouge()
    scores = rouge.get_scores(tokenized_hypothesis, tokenized_reference)
    return scores

# 사용 예시
label = "고양이가 매트 위에 앉아 있다."
bart_prediction = "매트 위에 고양이가 앉아 있다."
t5_prediction = "고양이가 매트 위에 앉아 있다."

rouge_scores = calculate_rouge_for_korean_sentence(label, bart_prediction)
print(rouge_scores[0]['rouge-l']['f'])
rouge_scores = calculate_rouge_for_korean_sentence(label, t5_prediction)
print(rouge_scores[0]['rouge-l']['f'])

0.7142857092857143
0.999999995


In [51]:
def calculate_rouge_for_korean_sentences(reference_sentences, hypothesis_sentences):
    assert len(reference_sentences) == len(hypothesis_sentences), "레이블과 예측 리스트의 길이가 같아야 합니다."

    # 형태소 분석기 초기화
    okt = Okt()

    # 형태소 분석을 통한 토큰화
    def tokenize(text):
        # 문자열이 아닌 경우 문자열로 변환
        if not isinstance(text, str):
            text = str(text)
        return ' '.join(okt.morphs(text))

    # ROUGE 계산기 초기화
    rouge = Rouge()

    # 모든 문장 쌍에 대해 ROUGE 점수 계산
    total_scores = {'rouge-1': {'f': 0.0}, 'rouge-2': {'f': 0.0}, 'rouge-l': {'f': 0.0}}
    for ref, hyp in zip(reference_sentences, hypothesis_sentences):
        tokenized_reference = tokenize(ref)
        tokenized_hypothesis = tokenize(hyp)

        scores = rouge.get_scores(tokenized_hypothesis, tokenized_reference)[0]
        for key in total_scores:
            total_scores[key]['f'] += scores[key]['f']

    # 평균 점수 계산
    avg_scores = {key: {'f': total_scores[key]['f'] / len(reference_sentences)} for key in total_scores}
    return avg_scores

In [42]:
# ckpt_950
qwen_part_labels = df['labels_고장부품'].tolist()
qwen_part_predictions = df['predictions_고장부품'].tolist()

qwen_part_avg_rouge_scores = calculate_rouge_for_korean_sentences(qwen_part_labels, qwen_part_predictions)
print("고장부품 :", qwen_part_avg_rouge_scores)

qwen_defect_labels = df['labels_불량유형'].tolist()
qwen_defect_predictions = df['predictions_불량유형'].tolist()

qwen_defect_avg_rouge_scores = calculate_rouge_for_korean_sentences(qwen_defect_labels, qwen_defect_predictions)
print("불량유형 :", qwen_defect_avg_rouge_scores)

qwen_action_labels = df['labels_조치내용'].tolist()
qwen_action_predictions = df['predictions_조치내용'].tolist()

qwen_action_avg_rouge_scores = calculate_rouge_for_korean_sentences(qwen_action_labels, qwen_action_predictions)
print("조치내용 :", qwen_action_avg_rouge_scores)

고장부품 : {'rouge-1': {'f': 0.8059124264286001}, 'rouge-2': {'f': 0.7362231941813137}, 'rouge-l': {'f': 0.8053469502381239}}
불량유형 : {'rouge-1': {'f': 0.615983550545878}, 'rouge-2': {'f': 0.28197164481128323}, 'rouge-l': {'f': 0.6151662428535701}}
조치내용 : {'rouge-1': {'f': 0.7708736800732328}, 'rouge-2': {'f': 0.6115488055179134}, 'rouge-l': {'f': 0.7605762863858689}}


In [32]:
# ckpt_1900
qwen_part_labels = df['labels_고장부품'].tolist()
qwen_part_predictions = df['predictions_고장부품'].tolist()

qwen_part_avg_rouge_scores = calculate_rouge_for_korean_sentences(qwen_part_labels, qwen_part_predictions)
print("고장부품 :", qwen_part_avg_rouge_scores)

qwen_defect_labels = df['labels_불량유형'].tolist()
qwen_defect_predictions = df['predictions_불량유형'].tolist()

qwen_defect_avg_rouge_scores = calculate_rouge_for_korean_sentences(qwen_defect_labels, qwen_defect_predictions)
print("불량유형 :", qwen_defect_avg_rouge_scores)

qwen_action_labels = df['labels_조치내용'].tolist()
qwen_action_predictions = df['predictions_조치내용'].tolist()

qwen_action_avg_rouge_scores = calculate_rouge_for_korean_sentences(qwen_action_labels, qwen_action_predictions)
print("조치내용 :", qwen_action_avg_rouge_scores)

고장부품 : {'rouge-1': {'f': 0.8306863772561595}, 'rouge-2': {'f': 0.7661019012992969}, 'rouge-l': {'f': 0.8298709010656832}}
불량유형 : {'rouge-1': {'f': 0.6430105343345569}, 'rouge-2': {'f': 0.2981787085089031}, 'rouge-l': {'f': 0.6423855343345568}}
조치내용 : {'rouge-1': {'f': 0.790219269260124}, 'rouge-2': {'f': 0.6184864520790896}, 'rouge-l': {'f': 0.782129374660306}}


In [52]:
# ckpt_2850
qwen_part_labels = df['labels_고장부품'].tolist()
qwen_part_predictions = df['predictions_고장부품'].tolist()

qwen_part_avg_rouge_scores = calculate_rouge_for_korean_sentences(qwen_part_labels, qwen_part_predictions)
print("고장부품 :", qwen_part_avg_rouge_scores)

qwen_defect_labels = df['labels_불량유형'].tolist()
qwen_defect_predictions = df['predictions_불량유형'].tolist()

qwen_defect_avg_rouge_scores = calculate_rouge_for_korean_sentences(qwen_defect_labels, qwen_defect_predictions)
print("불량유형 :", qwen_defect_avg_rouge_scores)

qwen_action_labels = df['labels_조치내용'].tolist()
qwen_action_predictions = df['predictions_조치내용'].tolist()

qwen_action_avg_rouge_scores = calculate_rouge_for_korean_sentences(qwen_action_labels, qwen_action_predictions)
print("조치내용 :", qwen_action_avg_rouge_scores)

고장부품 : {'rouge-1': {'f': 0.8547008422415757}, 'rouge-2': {'f': 0.79028047035382}, 'rouge-l': {'f': 0.8542585345492681}}
불량유형 : {'rouge-1': {'f': 0.6702821787711276}, 'rouge-2': {'f': 0.3272218787941121}, 'rouge-l': {'f': 0.6695678930568418}}
조치내용 : {'rouge-1': {'f': 0.8169502746880096}, 'rouge-2': {'f': 0.6662243811175296}, 'rouge-l': {'f': 0.8087578609788789}}
